In [ ]:
%matplotlib qt

import mne
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy import stats
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from statsmodels.stats.anova import AnovaRM


#### Load the recordings

We load every different recordings file from all individuals and we eventually tore the data in a specific structure that has all different epochs for different frequency bands. The number of epochs corresponds to the number of the subjects.

In [ ]:
# Directory containing .fif files
data_dir = 'recordings/fif_files'

# Frequency bands definition
bands = {
    'delta': (0, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

# Initialize dictionaries to store epochs for each band
band_epochs = {band: [] for band in bands}


In [ ]:
#This loads all data from all subjects and groups the different frequency bands

# Loop through all .fif files in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.fif'):
        filepath = os.path.join(data_dir, filename)
        
        # Load raw data
        raw = mne.io.read_raw_fif(filepath, preload=True)
        
        # Find events
        events = mne.find_events(raw)
        
        # Create epochs
        epochs = mne.Epochs(raw, events, event_id={'song_start': 3}, 
                            tmin=-30.0, tmax=120.0, baseline=(-30.0, 0), 
                            preload=True)
        
        # Loop through each band
        for band, (fmin, fmax) in bands.items():
            # Find the index of the epoch that corresponds to this band
            # Assuming the order of stimuli is always delta, theta, alpha, beta, gamma
            band_index = list(bands.keys()).index(band)
            
            if len(epochs) > band_index:
                # Extract the epoch for this band
                band_epoch = epochs[band_index]
                
                # Append to the corresponding band list
                band_epochs[band].append(band_epoch)

The data will be stored in the band_epochs[frequency_band] structure and can be accessed by band_epochs['delta'], band_epochs['theta'], etc.

In [ ]:
# Combine epochs for each band
for band in bands:
    if band_epochs[band]:
        band_epochs[band] = mne.concatenate_epochs(band_epochs[band])
    else:
        print(f"No epochs found for {band} band")

# Now band_epochs contains combined epochs for each frequency band across all subjects

#### Plot in time domain the data

We can use the mne plot method to plot the epochs time domain by commenting / uncommenting the following commands

In [ ]:
#plot the epochs of specific band

#band_epochs['delta'].plot(events=True)
#band_epochs['theta'].plot(events=True)
#band_epochs['alpha'].plot(events=True)
#band_epochs['beta'].plot(events=True)
#band_epochs['gamma'].plot(events=True)


#### Power Spectral Density (PSD) estimation and plots in different bands

Since we are mainly interested in analyzing the behaviour of brain during specific binaural sound in the correspondng frequency band, we calculate the Power Spectral Density of the data.
The resulting plots show the PSD of the data AVERAGED across all subjects to rule out any other activity of brain.

In [ ]:
tmin, tmax = 0.0, 10.0 #time to use in analysis and the estimation of the PSD (no need for baseline here)
fmin, fmax = 1.0, 48.0 #frequencies to show in analysis
win = 1 # this is the window length used in the PSD estimation
sfreq = epochs.info["sfreq"]
channels = ['C3', 'C4']

bands = {
    'delta': (0, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}


# Compute PSD in delta band
this_spectrum = band_epochs['delta'].compute_psd(
        "welch",
        picks = channels,
        n_fft=int(sfreq * win),
        n_overlap=0,
        n_per_seg=None,
        tmin=0,
        tmax=tmax,
        window="boxcar",
        fmin=fmin,
        fmax=fmax,
        verbose=False,
    )
psds, freqs = this_spectrum.get_data(return_freqs=True)
psd_mean = psds.mean(axis=0)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(freqs, 10 * np.log10(psd_mean.T))  # Convert to dB
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (dB/Hz)')
plt.title('PSD of Delta Band Epochs (Averaged across epochs and subjects)')
plt.xlim(fmin, fmax)
plt.grid(True)

# Add vertical lines to mark the specific band
f1,f2 = bands['delta']
plt.axvline(f1, color='r', linestyle='--', alpha=0.5)
plt.axvline(f2, color='r', linestyle='--', alpha=0.5)
plt.fill_between(bands['delta'], plt.ylim()[0], plt.ylim()[1], color='r', alpha=0.1)

# Add legend
plt.legend(channels)

plt.tight_layout()
plt.show()


# Compute PSD in theta band
this_spectrum = band_epochs['theta'].compute_psd(
        "welch",
        picks = channels,
        n_fft=int(sfreq * win),
        n_overlap=0,
        n_per_seg=None,
        tmin=0,
        tmax=tmax,
        window="boxcar",
        fmin=fmin,
        fmax=fmax,
        verbose=False,
    )
psds, freqs = this_spectrum.get_data(return_freqs=True)
psd_mean = psds.mean(axis=0)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(freqs, 10 * np.log10(psd_mean.T))  # Convert to dB
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (dB/Hz)')
plt.title('PSD of Theta Band Epochs (Averaged across epochs and subjects)')
plt.xlim(fmin, fmax)
plt.grid(True)

# Add vertical lines to mark the specific band
f1,f2 = bands['theta']
plt.axvline(f1, color='r', linestyle='--', alpha=0.5)
plt.axvline(f2, color='r', linestyle='--', alpha=0.5)
plt.fill_between(bands['theta'], plt.ylim()[0], plt.ylim()[1], color='r', alpha=0.1)

# Add legend
plt.legend(channels)

plt.tight_layout()
plt.show()

# Compute PSD in alpha band
this_spectrum = band_epochs['alpha'].compute_psd(
        "welch",
        picks = channels,
        n_fft=int(sfreq * win),
        n_overlap=0,
        n_per_seg=None,
        tmin=0,
        tmax=tmax,
        window="boxcar",
        fmin=fmin,
        fmax=fmax,
        verbose=False,
    )
psds, freqs = this_spectrum.get_data(return_freqs=True)
psd_mean = psds.mean(axis=0)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(freqs, 10 * np.log10(psd_mean.T))  # Convert to dB
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (dB/Hz)')
plt.title('PSD of Alpha Band Epochs (Averaged across epochs and subjects)')
plt.xlim(fmin, fmax)
plt.grid(True)

# Add vertical lines to mark the specific band
f1,f2 = bands['alpha']
plt.axvline(f1, color='r', linestyle='--', alpha=0.5)
plt.axvline(f2, color='r', linestyle='--', alpha=0.5)
plt.fill_between(bands['alpha'], plt.ylim()[0], plt.ylim()[1], color='r', alpha=0.1)

# Add legend
plt.legend(channels)

plt.tight_layout()
plt.show()


# Compute PSD in beta band
this_spectrum = band_epochs['beta'].compute_psd(
        "welch",
        picks = channels,
        n_fft=int(sfreq * win),
        n_overlap=0,
        n_per_seg=None,
        tmin=0,
        tmax=tmax,
        window="boxcar",
        fmin=fmin,
        fmax=fmax,
        verbose=False,
    )
psds, freqs = this_spectrum.get_data(return_freqs=True)
psd_mean = psds.mean(axis=0)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(freqs, 10 * np.log10(psd_mean.T))  # Convert to dB
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (dB/Hz)')
plt.title('PSD of Beta Band Epochs (Averaged across epochs and subjects)')
plt.xlim(fmin, fmax)
plt.grid(True)

# Add vertical lines to mark the specific band
f1,f2 = bands['beta']
plt.axvline(f1, color='r', linestyle='--', alpha=0.5)
plt.axvline(f2, color='r', linestyle='--', alpha=0.5)
plt.fill_between(bands['beta'], plt.ylim()[0], plt.ylim()[1], color='r', alpha=0.1)

# Add legend
plt.legend(channels)

plt.tight_layout()
plt.show()


# Compute PSD in gamma band
this_spectrum = band_epochs['gamma'].compute_psd(
        "welch",
        picks = channels,
        n_fft=int(sfreq * win),
        n_overlap=0,
        n_per_seg=None,
        tmin=0,
        tmax=tmax,
        window="boxcar",
        fmin=fmin,
        fmax=fmax,
        verbose=False,
    )
psds, freqs = this_spectrum.get_data(return_freqs=True)
psd_mean = psds.mean(axis=0)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(freqs, 10 * np.log10(psd_mean.T))  # Convert to dB
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (dB/Hz)')
plt.title('PSD of Gamma Band Epochs (Averaged across epochs and subjects)')
plt.xlim(fmin, fmax)
plt.grid(True)

# Add vertical lines to mark the specific band
f1,f2 = bands['gamma']
plt.axvline(f1, color='r', linestyle='--', alpha=0.5)
plt.axvline(f2, color='r', linestyle='--', alpha=0.5)
plt.fill_between(bands['gamma'], plt.ylim()[0], plt.ylim()[1], color='r', alpha=0.1)

# Add legend
plt.legend(channels)

plt.tight_layout()
plt.show()

#### Estimate and plot Time-Frequency Plots before and during each binaural sound 

- For TFR analysis, we need to set the baseline time interval in seconds. This will be used to compare the power therein with the power during the sound play. We don't care about the absolute values, we care about the change in the power in specific bands before and during the binaural sound.
- We can play around with these values. Which values are considered ideal for baseline? Which is the duration to be considered for the analysis that corresponds to the binaural sound play?
- Need we consider all the duration of the binaural sound play (2min -> 120secs) or just a small portion at the beginning of the play?

This cell plots 5 figures that show for each frequency band (delta, theta, alpha, beta, gamma) the percentage change in power for different frequencies and time periods.

In [ ]:
# need to set the times for the analysis. Minus times show time BEFORE the start of the sound.
baseline_tmin,baseline_tmax = -10.0,-5.0 # consider as baseline the time interval [-10.0 - -5.0]sec

# Define frequency ranges for each band
freq_ranges = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

# Create a figure with 2 rows and 3 columns of subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Time-Frequency Representations for Different Frequency Bands', fontsize=16)

# Flatten the axes array for easier indexing
axes = axes.flatten()

def plot_time_frequency(epochs, band_name, fmin, fmax, ax):
    print(f"Processing {band_name} band...")
    print(f"Number of epochs: {len(epochs)}")
    print(f"Epoch duration: {epochs.times[-1] - epochs.times[0]} seconds")
    print(f"Number of channels: {len(epochs.ch_names)}")
    
    # Define frequencies for analysis
    frequencies = np.logspace(*np.log10([fmin, fmax]), num=20)
    
    # Compute time-frequency representation
    power = epochs.compute_tfr(
        method="morlet",
        freqs=frequencies,
        n_cycles=frequencies / 2.,
        use_fft=True,
        return_itc=False,
        decim=3,
        n_jobs=1,
        average=True
    )
    
    # Print some information about the computed power
    print(f"Power shape: {power.data.shape}")
    print(f"Power min: {power.data.min()}, max: {power.data.max()}")
    
    # Apply baseline correction manually
   
    baseline = power.copy().crop(tmin=baseline_tmin, tmax=baseline_tmax)
    power.data = (power.data - baseline.data.mean(axis=-1, keepdims=True)) / baseline.data.mean(axis=-1, keepdims=True) * 100
    
    print(f"After baseline correction - min: {power.data.min()}, max: {power.data.max()}")
    
    # Plot manually
    mesh = ax.pcolormesh(power.times, frequencies, power.data.mean(axis=0),
                         cmap='RdBu_r', norm=colors.SymLogNorm(linthresh=0.05, linscale=0.05,
                                                              vmin=-100, vmax=100))
    ax.set_yscale('log')
    ax.set_title(f'{band_name.capitalize()}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Frequency (Hz)')
    
    # Add colorbar
    cbar = plt.colorbar(mesh, ax=ax)
    cbar.set_label('Power change (%)', rotation=270, labelpad=15)
    
    print(f"Plot created for {band_name} band")

# Perform time-frequency analysis for each band
for i, (band, epochs) in enumerate(band_epochs.items()):
    print(f"\nProcessing band {i+1}/5: {band}")
    fmin, fmax = freq_ranges[band]
    try:
        plot_time_frequency(epochs, band, fmin, fmax, axes[i])
        
        # Check for NaN or infinite values
        if np.any(np.isnan(epochs.get_data())) or np.any(np.isinf(epochs.get_data())):
            print(f"Warning: NaN or infinite values found in {band} band data")
        
    except Exception as e:
        print(f"Error processing {band} band: {str(e)}")
        axes[i].text(0.5, 0.5, f"Error processing {band} band:\n{str(e)}", 
                     ha='center', va='center', transform=axes[i].transAxes)

    # Set y-label for the leftmost subplots
    if i % 3 == 0:
        axes[i].set_ylabel('Frequency (Hz)')

    # Set x-label for the bottom subplots
    if i >= 3:
        axes[i].set_xlabel('Time (s)')

# Remove the empty subplot
fig.delaxes(axes[-1])

# Adjust layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

#### Compare the change in power bands for the mean epochs (mean across all subjects)

In the following we estimate the mean of the power for all subjects and we calculate the change of power in every different band.
The results are shown in two different bar plots, one for each different channel (C3, C4). The change in power is estimated as previously, separately in each frequency band taking into account the mean power of all the subjects and we compare the power in the baseline with regard to the power during the binaural sound.

In [ ]:

# Here we determine the duration of the analysis
baseline_tmin, baseline_tmax = -20, -10 # these define the start/end of the baseline 
sound_tmin, sound_tmax = 0, 20 # these define the start/end of the sound interval

# Define frequency ranges for each band
freq_ranges = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

def compute_power_difference(epochs, band_name, fmin, fmax, channels, baseline_tmin, baseline_tmax, sound_tmin, sound_tmax ):
    print(f"Processing {band_name} band for channels {channels}...")
    print(f"Number of epochs: {len(epochs)}")
    print(f"Epoch duration: {epochs.times[-1] - epochs.times[0]} seconds")
    
    channel_changes = {}
    
    for channel in channels:
        # Select the specific channel
        epochs_channel = epochs.copy().pick_channels([channel])
        
        # Define frequencies for analysis
        frequencies = np.linspace(fmin, fmax, num=20)
        
        # Compute time-frequency representation
        power = epochs_channel.compute_tfr(
            method="morlet",
            freqs=frequencies,
            n_cycles=frequencies / 2.,
            use_fft=True,
            return_itc=False,
            decim=3,
            n_jobs=1,
            average=True
        )
        
        
        # Calculate mean power in baseline period (-2 to 0 seconds)
        baseline_power = np.mean(power.data[:, :, (power.times >= baseline_tmin) & (power.times < baseline_tmax)])
        
        # Calculate mean power during binaural sound (0 to end)
        sound_power = np.mean(power.data[:, :, (power.times >= sound_tmin) & (power.times < sound_tmax) ])
        
        # Calculate percentage change
        power_change = (sound_power - baseline_power) / baseline_power * 100
        
        channel_changes[channel] = power_change
    
    # Calculate mean change across channels
    mean_change = np.mean(list(channel_changes.values()))
    
    print(f"Mean power change for {band_name}: {mean_change:.2f}%")
    
    return mean_change, channel_changes

# Compute power differences for each band
channels = ['C3', 'C4']
power_differences = {'Mean': [], 'C3': [], 'C4': []}
band_names = []

for band, epochs in band_epochs.items():
    fmin, fmax = freq_ranges[band]
    band_names.append(band)
    mean_change, channel_changes = compute_power_difference(epochs, band, fmin, fmax, channels,baseline_tmin, baseline_tmax,sound_tmin, sound_tmax)
    power_differences['Mean'].append(mean_change)
    power_differences['C3'].append(channel_changes['C3'])
    power_differences['C4'].append(channel_changes['C4'])

# Create bar plot
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))

x = np.arange(len(band_names))
width = 0.25

for i, (key, ax) in enumerate(zip(['Mean', 'C3', 'C4'], (ax1, ax2, ax3))):
    bars = ax.bar(x + i*width, power_differences[key], width, label=key)
    ax.set_ylabel('Power Change (%)')
    ax.set_title(f'Power Change During Binaural Sound Compared to Baseline - {key}')
    ax.set_xticks(x + width)
    ax.set_xticklabels(band_names)
    ax.legend()
    ax.axhline(y=0, color='r', linestyle='--')

    # Add value labels on top of each bar
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Print the results
for key in ['Mean', 'C3', 'C4']:
    print(f"\nResults for {key}:")
    for band, power_diff in zip(band_names, power_differences[key]):
        print(f"{band}: {power_diff:.2f}%")

#### Estimate the change in power (in the different bands) for every subject individually

Here we don't average the data from all subjects, but we want to see what happens for every single subject. So we estimate the power in the baseline, during the binaural sound and find the percentage change for each frequency band, channel and subject.
The results are stored in a structure power_differences[band][channel] for the 5 different bands and 2 different channels.

In [ ]:
# Here we determine the duration of the analysis
baseline_tmin, baseline_tmax = -20, -10 # these define the start/end of the baseline 
sound_tmin, sound_tmax = 0, 20 # these define the start/end of the sound interval

# Define frequency ranges for each band
freq_ranges = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

def compute_power_difference(epochs, band_name, fmin, fmax, channels, baseline_tmin, baseline_tmax, sound_tmin, sound_tmax):
    print(f"Processing {band_name} band for channels {channels}...")
    print(f"Number of epochs: {len(epochs)}")
    print(f"Epoch duration: {epochs.times[-1] - epochs.times[0]} seconds")
    
    channel_changes = {}
    epoch_changes = {}
    baseline_power_ind = {}
    sound_power_ind = {}

    for channel in channels:
        # Select the specific channel
        epochs_channel = epochs.copy().pick_channels([channel])
        
        # Define frequencies for analysis
        frequencies = np.linspace(fmin, fmax, num=20)
        
        # Compute time-frequency representation
        power = epochs_channel.compute_tfr(
            method="morlet",
            freqs=frequencies,
            n_cycles=frequencies / 2.,
            use_fft=True,
            return_itc=False,
            decim=3,
            n_jobs=1,
            average=False
        )
        
        tmin, tmax = -10, 10
        
        # Calculate mean power in baseline period (-20 to 0 seconds) for each epoch
        baseline_power = np.mean(power.data[:, :, :, (power.times >= baseline_tmin) & (power.times < baseline_tmax)], axis=(1, 2, 3))
        
        # Calculate mean power during binaural sound (0 to end) for each epoch
        sound_power = np.mean(power.data[:, :, :, (power.times >= sound_tmin) & (power.times < sound_tmax)], axis=(1, 2, 3))
        
        # Calculate percentage change for each epoch
        power_change = (sound_power - baseline_power) / baseline_power * 100
        
        channel_changes[channel] = np.mean(power_change)
        epoch_changes[channel] = power_change.tolist()
        baseline_power_ind[channel]=baseline_power.tolist()
        sound_power_ind[channel]=sound_power.tolist()

    
    # Calculate mean change across channels
    mean_change = np.mean(list(channel_changes.values()))
    
    print(f"Mean power change for {band_name}: {mean_change:.2f}%")
    
    return mean_change, channel_changes, epoch_changes,baseline_power_ind, sound_power_ind

# Usage in the main loop:
for band, epochs in band_epochs.items():
    fmin, fmax = freq_ranges[band]
    band_names.append(band)
    mean_change, channel_changes, epoch_changes, baseline_changes, sound_changes = compute_power_difference(epochs, band, fmin, fmax, channels, baseline_tmin, baseline_tmax, sound_tmin, sound_tmax)
    power_differences['Mean'].append(mean_change)
    power_differences['C3'].append(channel_changes['C3'])
    power_differences['C4'].append(channel_changes['C4'])
    
    # Store epoch-specific changes
    for channel in channels:
        if band not in power_differences:
            power_differences[band] = {}
        power_differences[band][channel] = epoch_changes[channel]

# Now power_differences[band][channel] contains a list of percentage changes for each epoch



#### Statistical Analysis
We use the changes of power in different channels (C3 and C4) as well as different frequency bands for every individual subject  and apply statistical analysis tools to see if there exist any statistically significant changes. 

We make two different plots:
- Figure 1: We depict with dots the power of each subject during baseline, binaural sound for channel C3 and C4. Normally we would expect to see a change (increase) of the power during the binaural sound compared to the baseline. Each dot corresponds to a subject (29 in total)
- Figure 2: Box-plots showing the change in power for all subjects in different bands (baseline and binaural sound)

In [ ]:
# Here we determine the duration of the analysis
baseline_tmin, baseline_tmax = -20, -10 # these define the start/end of the baseline 
sound_tmin, sound_tmax = 0, 20 # these define the start/end of the sound interval


# Define frequency ranges for each band
freq_ranges = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}

def compute_power_difference(epochs, band_name, fmin, fmax, channels, baseline_tmin, baseline_tmax, sound_tmin, sound_tmax):
    print(f"Processing {band_name} band for channels {channels}...")
    print(f"Number of epochs: {len(epochs)}")
    print(f"Epoch duration: {epochs.times[-1] - epochs.times[0]} seconds")
    
    channel_changes = {}
    epoch_changes = {}
    baseline_power_ind = {}
    sound_power_ind = {}
    
    for channel in channels:
        # Select the specific channel
        epochs_channel = epochs.copy().pick_channels([channel])
        
        # Define frequencies for analysis
        frequencies = np.linspace(fmin, fmax, num=20)
        
        # Compute time-frequency representation
        power = epochs_channel.compute_tfr(
            method="morlet",
            freqs=frequencies,
            n_cycles=frequencies / 2.,
            use_fft=True,
            return_itc=False,
            decim=3,
            n_jobs=1,
            average=False
        )
        
        tmin, tmax = -10, 10
        
        # Calculate mean power in baseline period (-20 to 0 seconds) for each epoch
        baseline_power = np.mean(power.data[:, :, :, (power.times >= baseline_tmin) & (power.times < baseline_tmax)], axis=(1, 2, 3))
        
        # Calculate mean power during binaural sound (0 to end) for each epoch
        sound_power = np.mean(power.data[:, :, :, (power.times >= sound_tmin) & (power.times < sound_tmax)], axis=(1, 2, 3))
        
        # Calculate percentage change for each epoch
        power_change = (sound_power - baseline_power) / baseline_power * 100
        
        channel_changes[channel] = np.mean(power_change)
        epoch_changes[channel] = power_change.tolist()
        baseline_power_ind[channel] = baseline_power.tolist()
        sound_power_ind[channel] = sound_power.tolist()
    
    # Calculate mean change across channels
    mean_change = np.mean(list(channel_changes.values()))
    print(f"Mean power change for {band_name}: {mean_change:.2f}%")
    
    return mean_change, channel_changes, epoch_changes, baseline_power_ind, sound_power_ind

# Initialize dictionaries to store results
power_differences = {'Mean': [], 'C3': [], 'C4': []}
baseline_powers = {}
sound_powers = {}
band_names = []

# Main loop
channels = ['C3', 'C4']
for band, epochs in band_epochs.items():
    fmin, fmax = freq_ranges[band]
    band_names.append(band)
    mean_change, channel_changes, epoch_changes, baseline_changes, sound_changes = compute_power_difference(epochs, band, fmin, fmax, channels, baseline_tmin, baseline_tmax, sound_tmin, sound_tmax)
    
    power_differences['Mean'].append(mean_change)
    power_differences['C3'].append(channel_changes['C3'])
    power_differences['C4'].append(channel_changes['C4'])
    
    # Store baseline and sound powers
    baseline_powers[band] = baseline_changes
    sound_powers[band] = sound_changes

# Create a DataFrame for statistical analysis
data = []
for band in band_names:
    for channel in channels:
        for i in range(len(baseline_powers[band][channel])):
            data.append({
                'Band': band,
                'Channel': channel,
                'Subject': i + 1,
                'Baseline_Power': baseline_powers[band][channel][i],
                'Sound_Power': sound_powers[band][channel][i]
            })

df = pd.DataFrame(data)

# Perform statistical analysis
print("\nStatistical Analysis:")
for band in band_names:
    for channel in channels:
        baseline = df[(df['Band'] == band) & (df['Channel'] == channel)]['Baseline_Power']
        sound = df[(df['Band'] == band) & (df['Channel'] == channel)]['Sound_Power']
        t_stat, p_value = stats.ttest_rel(baseline, sound)
        print(f"{band} - {channel}: t = {t_stat:.4f}, p = {p_value:.4f}")

# Visualization
plt.figure(figsize=(15, 10))
for i, band in enumerate(band_names):
    plt.subplot(2, 3, i+1)
    for channel in channels:
        baseline = df[(df['Band'] == band) & (df['Channel'] == channel)]['Baseline_Power']
        sound = df[(df['Band'] == band) & (df['Channel'] == channel)]['Sound_Power']
        plt.scatter(range(1, len(baseline) + 1), baseline, label=f'{channel} Baseline')
        plt.scatter(range(1, len(sound) + 1), sound, label=f'{channel} Sound')
    plt.title(f'{band} Band')
    plt.xlabel('Subject')
    plt.ylabel('Power')
    plt.legend()
plt.tight_layout()
plt.show()

# Box plot
plt.figure(figsize=(15, 10))
df_melted = df.melt(id_vars=['Band', 'Channel', 'Subject'], 
                    value_vars=['Baseline_Power', 'Sound_Power'],
                    var_name='Condition', value_name='Power')
sns.boxplot(x='Band', y='Power', hue='Condition', data=df_melted)
plt.title('Baseline vs Sound Power Across Bands')
plt.xticks(rotation=45)
plt.show()

#### Statistical tests (continued)

In this section we perform some extra statistical tests and in particular:

- we perform repeated ANOVA statistical tests of the power regarding to baseline / binaural sound for different bands, channels and subject.
- We apply the mixed-effects model
- we apply Pairwise comparisons (Baseline vs Sound) for all different frequency bands and channels.
- we estimate the Effect size calculation (Cohen's d)

In [ ]:


# First, let's reshape the data for easier analysis
df_long = df.melt(id_vars=['Band', 'Channel', 'Subject'], 
                  value_vars=['Baseline_Power', 'Sound_Power'],
                  var_name='Condition', value_name='Power')
df_long['Condition'] = df_long['Condition'].map({'Baseline_Power': 'Baseline', 'Sound_Power': 'Sound'})

# 1. Repeated Measures ANOVA
aov = AnovaRM(data=df_long, depvar='Power', subject='Subject', 
              within=['Band', 'Channel', 'Condition'])
res_aov = aov.fit()
print("Repeated Measures ANOVA:")
print(res_aov.summary())

# 2. Mixed-Effects Model
model = mixedlm("Power ~ Band + Channel + Condition + Band:Channel + Band:Condition + Channel:Condition", 
                data=df_long, groups="Subject")
res_mix = model.fit()
print("\nMixed-Effects Model:")
print(res_mix.summary())

# 3. Pairwise comparisons for each Band and Channel
print("\nPairwise comparisons (Baseline vs Sound):")
for band in df['Band'].unique():
    for channel in df['Channel'].unique():
        baseline = df[(df['Band'] == band) & (df['Channel'] == channel)]['Baseline_Power']
        sound = df[(df['Band'] == band) & (df['Channel'] == channel)]['Sound_Power']
        t_stat, p_val = stats.ttest_rel(baseline, sound)
        print(f"{band} - {channel}: t = {t_stat:.4f}, p = {p_val:.4f}")

# 4. Visualization
plt.figure(figsize=(15, 10))
sns.boxplot(x='Band', y='Power', hue='Condition', data=df_long)
plt.title('Power Distribution by Band and Condition')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 5. Effect size calculation (Cohen's d)
print("\nEffect sizes (Cohen's d):")
for band in df['Band'].unique():
    for channel in df['Channel'].unique():
        baseline = df[(df['Band'] == band) & (df['Channel'] == channel)]['Baseline_Power']
        sound = df[(df['Band'] == band) & (df['Channel'] == channel)]['Sound_Power']
        d = (np.mean(sound) - np.mean(baseline)) / np.sqrt((np.std(sound)**2 + np.std(baseline)**2) / 2)
        print(f"{band} - {channel}: d = {d:.4f}")

# 6. Interaction plot
plt.figure(figsize=(15, 10))
for channel in df['Channel'].unique():
    channel_data = df_long[df_long['Channel'] == channel]
    sns.lineplot(x='Band', y='Power', hue='Condition', data=channel_data, marker='o')
    plt.title(f'Interaction Plot - {channel}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

#### Boxplots of baseline / sound period in different channels / bands

Here we show a plot with the boxplots of baseline / sound period in different channels / bands

In [ ]:

# Reshape the data for easier plotting
df_long = df.melt(id_vars=['Band', 'Channel', 'Subject'], 
                  value_vars=['Baseline_Power', 'Sound_Power'],
                  var_name='Condition', value_name='Power')
df_long['Condition'] = df_long['Condition'].map({'Baseline_Power': 'Baseline', 'Sound_Power': 'Sound'})

# Set up the plot
plt.figure(figsize=(20, 12))

# Create a box plot for each channel
for i, channel in enumerate(['C3', 'C4']):
    plt.subplot(2, 1, i+1)
    
    # Create the box plot
    sns.boxplot(x='Band', y='Power', hue='Condition', 
                data=df_long[df_long['Channel'] == channel],
                palette='Set3')
    
    # Customize the plot
    plt.title(f'Power Distribution by Frequency Band - Channel {channel}', fontsize=16)
    plt.xlabel('Frequency Band', fontsize=12)
    plt.ylabel('Power', fontsize=12)
    plt.legend(title='Condition', fontsize=10, title_fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

# Create a combined box plot for all data
plt.figure(figsize=(20, 10))
sns.boxplot(x='Band', y='Power', hue='Condition', 
            data=df_long, palette='Set3')
plt.title('Power Distribution by Frequency Band - All Channels', fontsize=16)
plt.xlabel('Frequency Band', fontsize=12)
plt.ylabel('Power', fontsize=12)
plt.legend(title='Condition', fontsize=10, title_fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

#### Save of individual analysis results 

In this section, we calculate per subject / channel / band the mean power in baseline, in sound and the percentage change. The results for all subjects are stored in the 'power_analysis_results.csv' file that has the following columns:

```
Subject,Baseline_Power_alpha_C3,Baseline_Power_alpha_C4,Baseline_Power_beta_C3,Baseline_Power_beta_C4,Baseline_Power_delta_C3,Baseline_Power_delta_C4,Baseline_Power_gamma_C3,Baseline_Power_gamma_C4,Baseline_Power_theta_C3,Baseline_Power_theta_C4,Percent_Change_alpha_C3,Percent_Change_alpha_C4,Percent_Change_beta_C3,Percent_Change_beta_C4,Percent_Change_delta_C3,Percent_Change_delta_C4,Percent_Change_gamma_C3,Percent_Change_gamma_C4,Percent_Change_theta_C3,Percent_Change_theta_C4,Sound_Power_alpha_C3,Sound_Power_alpha_C4,Sound_Power_beta_C3,Sound_Power_beta_C4,Sound_Power_delta_C3,Sound_Power_delta_C4,Sound_Power_gamma_C3,Sound_Power_gamma_C4,Sound_Power_theta_C3,Sound_Power_theta_C4
```

In [ ]:

# empty list to store the results
results = []

# Iterate through each subject
for subject in df['Subject'].unique():
    subject_data = df[df['Subject'] == subject]
    
    # Iterate through each band and channel
    for band in subject_data['Band'].unique():
        for channel in subject_data['Channel'].unique():
            data = subject_data[(subject_data['Band'] == band) & (subject_data['Channel'] == channel)]
            
            baseline_power = data['Baseline_Power'].values[0]
            sound_power = data['Sound_Power'].values[0]
            
            # Calculate percentage change
            percent_change = ((sound_power - baseline_power) / baseline_power) * 100
            
            results.append({
                'Subject': subject,
                'Band': band,
                'Channel': channel,
                'Baseline_Power': baseline_power,
                'Sound_Power': sound_power,
                'Percent_Change': percent_change
            })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Pivot the table to have bands and channels as columns
pivot_df = results_df.pivot_table(
    index='Subject', 
    columns=['Band', 'Channel'], 
    values=['Baseline_Power', 'Sound_Power', 'Percent_Change']
)

# Flatten column multi-index
pivot_df.columns = [f'{val[0]}_{val[1]}_{val[2]}' for val in pivot_df.columns]

# Reset index to make Subject a column
pivot_df = pivot_df.reset_index()

# Sort columns for better readability
column_order = ['Subject'] + sorted(pivot_df.columns[1:])
pivot_df = pivot_df[column_order]

# Display the first few rows of the table
print(pivot_df.head())

# Save the full table to a CSV file
pivot_df.to_csv('power_analysis_results.csv', index=False)

print("\nFull results table has been saved to 'power_analysis_results.csv'")